In [6]:
import cv2
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import base64
import os
import time
import numpy as np

# Define the Tesla model to search and the number of images you want
model = "Model_3"
num_of_images = 20  # Set how many images you want to download
download_folder = f"dataset/{model}_download"

# YOLOv4 paths
weights_path = "../yolov4/yolov4.weights"
config_path = "../yolov4/yolov4.cfg"
coco_names_path = "../yolov4/coco.names"

# Load YOLOv4 network
try:
    net = cv2.dnn.readNet(weights_path, config_path)
    with open(coco_names_path, "r") as f:
        classes = f.read().strip().split("\n")
except Exception as e:
    print(f"Error loading YOLO files: {e}")
    exit()

# Define class of interest
target_class = "car"

# Initialize WebDriver
driver = webdriver.Chrome()  # Ensure ChromeDriver is installed
driver.get("https://images.google.com/")

try:
    # Enter your search
    search_query = f"Tesla {model}"
    search_box = driver.find_element(By.NAME, "q")
    search_box.send_keys(search_query + "\n")
    time.sleep(2)  # Allow some time to load

    # Create the download folder if it doesn't exist
    os.makedirs(download_folder, exist_ok=True)

    # Function to detect a car in an image using YOLOv4
    def contains_car(image_data):
        image = cv2.imdecode(np.frombuffer(image_data, np.uint8), cv2.IMREAD_COLOR)
        if image is None:
            return False

        blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
        net.setInput(blob)
        layer_names = net.getLayerNames()
        output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

        detections = net.forward(output_layers)
        for detection in detections:
            for object_data in detection:
                scores = object_data[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if classes[class_id] == target_class and confidence > 0.5:
                    return True
        return False

    # Loop to scroll, load, and download images until reaching num_of_images
    successful_downloads = 0
    scroll_attempts = 0
    while successful_downloads < num_of_images and scroll_attempts < 10:
        # Scroll to load more images
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Allow some time to load
        scroll_attempts += 1

        # Get new image URLs
        soup = BeautifulSoup(driver.page_source, "html.parser")
        image_elements = soup.find_all("img")

        # Download images that contain cars
        for idx, img in enumerate(image_elements):
            img_url = img.get("src")
            if img_url:
                # Handle base64 or direct URLs
                if img_url.startswith("data:image"):
                    base64_data = img_url.split(",")[1]
                    img_data = base64.b64decode(base64_data)
                else:
                    img_data = requests.get(img_url).content

                # Check if image contains a car
                if contains_car(img_data):
                    file_path = os.path.join(download_folder, f"img_{successful_downloads+1}.jpg")
                    with open(file_path, "wb") as img_file:
                        img_file.write(img_data)
                    print(f"Downloaded img_{successful_downloads+1}.jpg containing a car")
                    successful_downloads += 1

                    # Stop if we've reached the desired number of downloads
                    if successful_downloads >= num_of_images:
                        break

            # Stop downloading after reaching the desired number of images
            if successful_downloads >= num_of_images:
                break

finally:
    driver.quit()

Downloaded img_0.jpg containing a car
Downloaded img_1.jpg containing a car
Downloaded img_2.jpg containing a car
Downloaded img_3.jpg containing a car
Downloaded img_4.jpg containing a car
Downloaded img_5.jpg containing a car
Downloaded img_6.jpg containing a car
Downloaded img_7.jpg containing a car
Downloaded img_8.jpg containing a car
Downloaded img_9.jpg containing a car
Downloaded img_10.jpg containing a car
Downloaded img_11.jpg containing a car
Downloaded img_12.jpg containing a car
Downloaded img_13.jpg containing a car
Downloaded img_14.jpg containing a car
Downloaded img_15.jpg containing a car
Downloaded img_16.jpg containing a car
Downloaded img_17.jpg containing a car
Downloaded img_18.jpg containing a car
Downloaded img_19.jpg containing a car
